In [3]:
from langchain.vectorstores import FAISS
from langchain.memory import VectorStoreRetrieverMemory
from langchain_core.prompts import PromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFaceEndpoint, ChatHuggingFace
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()

True

In [4]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
memory_store = FAISS.from_texts([""], embeddings)
memory = VectorStoreRetrieverMemory(retriever=memory_store.as_retriever())

d:\anaconda3\envs\genai_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\pc\AppData\Local\Temp\ipykernel_4540\3823595349.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = VectorStoreRetrieverMemory(retriever=memory_store.as_retriever())


In [5]:
prompt_template = """
You are a helpful assistant.

Conversation history (retrieved from memory):
{history}

User Query: {query}

Take the context of history conversation if necessary, if not then answer the query as it is.
Don't overexplain things.
"""

In [6]:
prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["history", "query"]
)

prompt

PromptTemplate(input_variables=['history', 'query'], input_types={}, partial_variables={}, template="\nYou are a helpful assistant.\n\nConversation history (retrieved from memory):\n{history}\n\nUser Query: {query}\n\nTake the context of history conversation if necessary, if not then answer the query as it is.\nDon't overexplain things.\n")

In [7]:
llm = HuggingFaceEndpoint(
    model="meta-llama/Llama-3.3-70B-Instruct",
    task="text-generation",
    temperature=0
)

model = ChatHuggingFace(llm=llm)

In [8]:
parser = StrOutputParser()

In [9]:
def get_history(query: str):
    history_docs = memory.load_memory_variables({'input':query})
    return history_docs.get("history", "")

In [10]:
parallel_chain = RunnableParallel({
    'history': RunnableLambda(get_history),
    'query': RunnablePassthrough()
})

In [11]:
main_chain = parallel_chain | prompt | model | parser

In [12]:
def ask(query: str):
    answer = main_chain.invoke(query)
    memory.save_context({"input": query}, {"output": answer})
    return answer

In [13]:
ask("can you explain me about the diffusion transformers in short?")

"Diffusion transformers are a type of deep learning model that combines diffusion models with transformer architectures. They're mainly used for image and video generation tasks, like image synthesis and editing. The diffusion part helps with generating new data by iteratively refining noise, while the transformer part helps with understanding the context and structure of the data."

In [14]:
ask("compare its architecture from the standard one.")

'Compared to the standard transformer architecture, diffusion transformers have an additional diffusion-based encoder and decoder. The standard transformer relies on self-attention mechanisms, whereas diffusion transformers combine this with a diffusion process that progressively refines the input noise to generate the output. This allows diffusion transformers to better handle image and video generation tasks.'

In [15]:
ask("Generate a detailed report on AI revolution in India.")

"The AI revolution in India has witnessed significant growth in recent years, with the country emerging as a prominent player in the global AI landscape. Here's a detailed report:\n\n**Introduction**\nThe AI revolution in India began to take shape in the early 2010s, with the government launching initiatives to promote AI research and development. Since then, the country has made tremendous progress, with AI being increasingly adopted across various sectors, including healthcare, finance, education, and transportation.\n\n**History and Milestones**\nIndia's AI journey can be divided into three phases:\n1. **Early beginnings (2010-2015)**: The government launched initiatives like the National e-Governance Plan and the Digital India program, which laid the foundation for AI adoption.\n2. **Growth phase (2015-2020)**: The government launched the National AI Strategy, which outlined a roadmap for AI development in India. This phase saw significant investments in AI research and development